<a href="https://colab.research.google.com/github/ogut77/MovieLensApacheSpark/blob/master/RecommenderMovie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://ftp.itu.edu.tr/Mirror/Apache/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [4]:
!ls

sample_data  spark-2.4.0-bin-hadoop2.7	spark-2.4.0-bin-hadoop2.7.tgz


In [5]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

lines = sc.textFile("sample_data") 
lines.take(5)
#number of lines
print(lines.count())

50068


In [6]:
import urllib.request
import pandas as pd 
import zipfile
import os
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

import os

small=urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-latest-small.zip','/content/drive/My Drive/small.zip')
big=urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-latest.zip','/content/drive/My Drive/big.zip')

with zipfile.ZipFile('/content/drive/My Drive/small.zip', "r") as z:
   z.extractall('/content/drive/My Drive/')
    
with zipfile.ZipFile('/content/drive/My Drive/big.zip', "r") as z:
   z.extractall('/content/drive/My Drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
!ls "/content/drive/My Drive"

 big.zip	    Iphone	  ml-latest	        ratings.csv
 Book1a.csv	    kaggle.json   ml-latest-small       small.zip
'Colab Notebooks'   mel		  paris_completed.wf1


In [0]:
small_ratings_raw_data = sc.textFile('/content/drive/My Drive/ml-latest-small/ratings.csv')
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

In [0]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [10]:
small_ratings_data.take(3)

[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

In [11]:
small_movies_raw_data = sc.textFile('/content/drive/My Drive/ml-latest-small/movies.csv')
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [0]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([0.6, 0.2, 0.2], seed=42)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [13]:
from pyspark.mllib.recommendation import ALS
import math

seed = 42
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print('For rank %s the RMSE is %s'+ str( (rank, error)))
    if error < min_error:
        min_error = error
        best_rank = rank

print('The best model was trained with rank %s'+ str( best_rank))

For rank %s the RMSE is %s(4, 0.9085735126459838)
For rank %s the RMSE is %s(8, 0.9093646061880156)
For rank %s the RMSE is %s(12, 0.9062301308911616)
The best model was trained with rank %s12


In [14]:
predictions.take(3)

[((140, 1084), 3.5153012095701484),
 ((74, 1084), 4.177394484005578),
 ((402, 1084), 3.9674954778809974)]

In [15]:
rates_and_preds.take(3)

[((1, 553), (5.0, 4.351966643985909)),
 ((1, 673), (3.0, 2.921655604127569)),
 ((1, 1025), (5.0, 4.302670516478489))]

In [16]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is '+str(error))

For testing data the RMSE is 0.9175771942446856


In [17]:
complete_ratings_raw_data = sc.textFile('/content/drive/My Drive/ml-latest/ratings.csv')
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print ("There are %s recommendations in the complete dataset" % (complete_ratings_data.count()))

There are 27753444 recommendations in the complete dataset


In [0]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=42)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, 
                           iterations=iterations, lambda_=regularization_parameter)

In [20]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print("For testing data the RMSE is %s" % (error))

For testing data the RMSE is 0.8159314421483166


In [21]:
complete_movies_raw_data = sc.textFile('/content/drive/My Drive/ml-latest/movies.csv')
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print("There are %s movies in the complete dataset" % (complete_movies_titles.count()))

There are 58098 movies in the complete dataset


In [0]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [23]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,4), # Star Wars (1977)
     (0,1,3), # Toy Story (1995)
     (0,16,3), # Casino (1995)
     (0,25,4), # Leaving Las Vegas (1995)
     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,1), # Flintstones, The (1994)
     (0,379,1), # Timecop (1994)
     (0,296,3), # Pulp Fiction (1994)
     (0,858,5) , # Godfather, The (1972)
     (0,50,4) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print("New user ratings: %s" % new_user_ratings_RDD.take(10))

New user ratings: [(0, 260, 4), (0, 1, 3), (0, 16, 3), (0, 25, 4), (0, 32, 4), (0, 335, 1), (0, 379, 1), (0, 296, 3), (0, 858, 5), (0, 50, 4)]


In [0]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

In [25]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print("New model trained in %s seconds" % round(tt,3))

New model trained in 321.917 seconds


In [0]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [103]:
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(6216,
  ((2.9929557560373636, 'Nowhere in Africa (Nirgendwo in Afrika) (2001)'),
   717)),
 (124320, ((2.996900165645125, 'Once a Thief (1965)'), 1)),
 (83916, ((2.880113618716198, 'Blues in the Night (1941)'), 9))]

In [0]:
#(Title, Rating, Ratings Count).
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))



In [105]:
new_user_recommendations_rating_title_and_count_RDD.take(3)

[('Nowhere in Africa (Nirgendwo in Afrika) (2001)', 2.9929557560373636, 717),
 ('Once a Thief (1965)', 2.996900165645125, 1),
 ('Blues in the Night (1941)', 2.880113618716198, 9)]

In [0]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])


In [107]:
print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
("Long Night's Journey Into Day (2000)", 4.008593124114109, 35)
('"Godfather', 3.9800732630420717, 60904)
('"Godfather: Part II', 3.884065604643434, 38875)
('Mikra Anglia (2013)', 3.8606531360739065, 27)
('"Woman In Berlin', 3.7893214294453226, 39)
('Pulp Fiction (1994)', 3.7861097008326174, 92406)
('Frozen Planet (2011)', 3.76823273272918, 402)
('Fish Story (Fisshu sutôrî) (2009)', 3.7572742663722223, 28)
('Cosmos', 3.7462802507335047, 157)
('"Lord of the Rings: The Fellowship of the Ring', 3.743100290258634, 61883)
('Heimat - A Chronicle of Germany (Heimat - Eine deutsche Chronik) (1984)', 3.7420182720725457, 35)
("Schindler's List (1993)", 3.741582210907684, 71516)
("Glen Campbell: I'll Be Me (2014)", 3.730441514318645, 30)
("One Flew Over the Cuckoo's Nest (1975)", 3.7256550790169403, 42181)
('"Lord of the Rings: The Return of the King', 3.7242841288040642, 57378)
('"Shawshank Redemption', 3.722610405802812, 97999)
('"Two Escobars

In [108]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=116688, rating=0.71559653175492)]